# Credit Risk Resampling Techniques

In [1]:
import warnings


warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from collections import Counter
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import ClusterCentroids
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
import pickle







# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('forlex.csv')
df = pd.read_csv(file_path)
df.tail(50)

,ID,LIMIT_BAL,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_pay,Total_Missed_Pymts,Payment_Ratio,Total_Bill_AMT,Total_Payment,%_Amount_Paid,Risk_Level
29950,29951,220000.0,1,32,0,1,0,0,0,0,...,500.0,0.0,264.0,0,1,16.666667,2140.0,1320.0,61.682243,Low
29951,29952,160000.0,2,30,0,0,0,0,0,0,...,5011.0,10006.0,8070.0,0,0,0.000000,569941.0,34368.0,6.030098,NaN
29952,29953,210000.0,2,30,0,0,0,0,0,0,...,0.0,626.0,626.0,0,0,0.000000,18199.0,13993.0,76.888840,NaN
29953,29954,170000.0,2,30,0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0.000000,250582.0,7615.0,3.038925,NaN
29954,29955,180000.0,2,30,0,0,0,0,1,0,...,66.0,57317.0,70052.0,0,1,16.666667,188301.0,132673.0,70.457937,Low
29955,29956,410000.0,2,32,0,1,0,0,0,0,...,0.0,1539.0,0.0,0,1,16.666667,2869.0,2109.0,73.509934,Low
29956,29957,150000.0,2,30,0,0,0,0,0,0,...,1000.0,3000.0,0.0,0,0,0.000000,18207.0,10000.0,54.923930,NaN
29957,29958,500000.0,2,31,0,0,0,0,0,0,...,7524.0,5009.0,2333.0,0,0,0.000000,805132.0,325463.0,40.423558,NaN
29958,29959,230000.0,2,31,0,0,0,0,0,0,...,1000.0,0.0,0.0,0,0,0.000000,5346.0,4000.0,74.822297,NaN
29959,29960,290000.0,2,32,0,0,0,0,0,0,...,18711.0,0.0,157.0,0,0,0.000000,190162.0,164422.0,86.464173,NaN


In [4]:
# Create the variable bins to define our bounds for when we cut the dataframe.
bins = [0.000000,16.666666, 33.333332,50.000000, 66.666666, 100.000000]

# Create names for the bins
group_names = ["None","Low", "Moderate", "Above Average", "High"]

In [5]:
df["Risk_Level"] = pd.cut(df["Payment_Ratio"], bins, labels=group_names)

df.head()

#df.to_csv('forlex.csv', encoding='utf-8', index=False)


,ID,LIMIT_BAL,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_pay,Total_Missed_Pymts,Payment_Ratio,Total_Bill_AMT,Total_Payment,%_Amount_Paid,Risk_Level
0,1,20000.0,1,24,1,1,0,0,0,0,...,0.0,0.0,0.0,1,2,33.333333,7704.0,689.0,8.943406,Moderate
1,2,120000.0,2,26,0,1,0,0,0,1,...,1000.0,0.0,2000.0,1,2,33.333333,17077.0,5000.0,29.279147,Moderate
2,3,90000.0,2,34,0,0,0,0,0,0,...,1000.0,1000.0,5000.0,0,0,0.000000,101653.0,11018.0,10.838834,NaN
3,4,50000.0,1,37,0,0,0,0,0,0,...,1100.0,1069.0,1000.0,0,0,0.000000,231334.0,8388.0,3.625926,NaN
4,5,50000.0,1,57,0,0,0,0,0,0,...,9000.0,689.0,679.0,0,0,0.000000,109339.0,59049.0,54.005433,NaN


In [6]:
values = {'Risk_Level': 'None'}
df=df.fillna(value=values)

In [7]:
df.head()

,ID,LIMIT_BAL,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_pay,Total_Missed_Pymts,Payment_Ratio,Total_Bill_AMT,Total_Payment,%_Amount_Paid,Risk_Level
0,1,20000.0,1,24,1,1,0,0,0,0,...,0.0,0.0,0.0,1,2,33.333333,7704.0,689.0,8.943406,Moderate
1,2,120000.0,2,26,0,1,0,0,0,1,...,1000.0,0.0,2000.0,1,2,33.333333,17077.0,5000.0,29.279147,Moderate
2,3,90000.0,2,34,0,0,0,0,0,0,...,1000.0,1000.0,5000.0,0,0,0.000000,101653.0,11018.0,10.838834,None
3,4,50000.0,1,37,0,0,0,0,0,0,...,1100.0,1069.0,1000.0,0,0,0.000000,231334.0,8388.0,3.625926,None
4,5,50000.0,1,57,0,0,0,0,0,0,...,9000.0,689.0,679.0,0,0,0.000000,109339.0,59049.0,54.005433,None


In [8]:
df.corr()['default_pay']


ID                   -0.013952
LIMIT_BAL            -0.153520
MARRIAGE             -0.024339
AGE                   0.013890
PAY_0                 0.368117
PAY_2                 0.338022
PAY_3                 0.293583
PAY_4                 0.275495
PAY_5                 0.266959
PAY_6                 0.246089
BILL_AMT1            -0.019644
BILL_AMT2            -0.014193
BILL_AMT3            -0.014076
BILL_AMT4            -0.010156
BILL_AMT5            -0.006760
BILL_AMT6            -0.005372
PAY_AMT1             -0.072929
PAY_AMT2             -0.058579
PAY_AMT3             -0.056250
PAY_AMT4             -0.056827
PAY_AMT5             -0.055124
PAY_AMT6             -0.053183
default_pay           1.000000
Total_Missed_Pymts    0.398394
Payment_Ratio         0.398394
Total_Bill_AMT       -0.012691
Total_Payment        -0.102354
%_Amount_Paid        -0.011391
Name: default_pay, dtype: float64

In [9]:
X=df[['MARRIAGE', 'AGE','PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6','Total_Missed_Pymts', 'Payment_Ratio','Total_Payment']]


In [10]:
y=df[['default_pay']]

# Split the Data into Training and Testing

In [11]:
# Check the balance of our target values

y['default_pay'].value_counts()

0    23364
1     6636
Name: default_pay, dtype: int64

In [12]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [13]:
print(f'''% Positive class in Train ={np.round(y_train.value_counts(normalize=True)[1] *100, 2)}''')

% Positive class in Train =default_pay
1              22.08
dtype: float64


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [14]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
# YOUR CODE HERE
scaler = StandardScaler()



In [15]:
 #Fit the Standard Scaler with the training data
 #When fitting scaling functions, only train on the training dataset
#YOUR CODE HERE
X_scaler = scaler.fit(X_train)


In [16]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


### Smoteen

In [17]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE

from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
smoteen_X_resampled, smoteen_y_resampled = sm.fit_resample(X_train_scaled, y_train)

pickle.dump(sm,open('smoteenn.pkl', 'wb'))

# View the count of target classes with Counter
# YOUR CODE HERE

In [18]:
Counter(y_train.value_counts())

Counter({17532: 1, 4968: 1})

In [19]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
smoteen_model= LogisticRegression(solver='lbfgs', random_state=1)
smoteen_model.fit(smoteen_X_resampled, smoteen_y_resampled)

LogisticRegression(random_state=1)

In [20]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
smoteen_y_pred = smoteen_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, smoteen_y_pred)

0.6969757527311484

In [21]:
print(f"Training Data Score: {smoteen_model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {smoteen_model.score(X_test_scaled, y_test)}")

Training Data Score: 0.7636444444444445
Testing Data Score: 0.7624


In [22]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, smoteen_y_pred)

array([[4752, 1080],
       [ 702,  966]], dtype=int64)

In [23]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, smoteen_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.87      0.81      0.58      0.84      0.69      0.48      5832
          1       0.47      0.58      0.81      0.52      0.69      0.46      1668

avg / total       0.78      0.76      0.63      0.77      0.69      0.48      7500

